In [2]:
import pandas as pd
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import mlflow
from mlflow.models import infer_signature



In [3]:
## set the tracking uri
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")

In [8]:
x,y=datasets.load_iris(return_X_y=True)

## split the data into training and test

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20)

## define the model hyperparameter
params = {"penalty":"l2","solver": "lbfgs", "max_iter": 1000, "multi_class": "auto", "random_state": 8888}

lr=LogisticRegression(**params)
lr.fit(x_train,y_train)




c:\python\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(max_iter=1000, multi_class='auto', random_state=8888)

In [9]:
## prediction on the test set

y_pred=lr.predict(x_test)
y_pred



array([1, 0, 2, 1, 2, 1, 0, 1, 1, 0, 0, 0, 1, 2, 0, 1, 2, 1, 0, 1, 2, 2,
       2, 1, 1, 1, 0, 2, 0, 2])

In [12]:
accuracy=accuracy_score(y_test,y_pred)
print(accuracy)



1.0


In [14]:
## mlflow tracking
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")

## create a new mlflow experiment
mlflow.set_experiment("MLFLOW Quickstart")

## start an Mlflow run
with mlflow.start_run():
    ## log the hyperparameters
    mlflow.log_params(params)
    ## log the accuracy metrics
    mlflow.log_metric("accuracy",accuracy)

    ## set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "Basic LR model for iris data")

    ## infer the model signature

    signature = infer_signature(x_train,lr.predict(x_train))

    ## log the model
    model_info=mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="iris_model",
        signature=signature,
        input_example=x_train,
        registered_model_name="tracking-quickstart"
    )

Successfully registered model 'tracking-quickstart'.
2025/06/01 13:32:53 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracking-quickstart, version 1


🏃 View run thundering-yak-949 at: http://127.0.0.1:5000/#/experiments/727529071752205176/runs/6bd9714286544a3fa546f47165af04f1
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/727529071752205176


Created version '1' of model 'tracking-quickstart'.


In [15]:
model_info.model_uri

'runs:/6bd9714286544a3fa546f47165af04f1/iris_model'

In [17]:
## load the model back for prediction as a generic python function model 

loaded_model=mlflow.pyfunc.load_model(model_info.model_uri)
predictions=loaded_model.predict(x_test)
iris_features_name=datasets.load_iris().feature_names

result=pd.DataFrame(x_test,columns=iris_features_name)

result["actual_class"]=y_test
result["predicted_class"]=predictions

In [19]:
result[:5]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),actual_class,predicted_class
0,6.5,2.8,4.6,1.5,1,1
1,4.8,3.4,1.9,0.2,0,0
2,7.1,3.0,5.9,2.1,2,2
3,6.6,2.9,4.6,1.3,1,1
4,5.8,2.8,5.1,2.4,2,2
